In [17]:
import os
os.chdir('/root/DiffusionBasedRL') 

In [18]:
# Change This
from config.maze2d_dit import base
config = base['diffusion']

In [19]:
from loguru import logger
import os
logger.info(os.getcwd())
import sys
logger.info(sys.executable)
sys.path.append(os.getcwd())
import numpy as np

from diffuser.models.dit import LDiT_models

import diffuser.utils as utils


#-----------------------------------------------------------------------------#
#----------------------------------- setup -----------------------------------#
#-----------------------------------------------------------------------------#

class argsmaker:
    # Change this
    dataset: str = 'maze2d-large-v1' #maze2d-large-v1, maze2d-medium-v1, maze2d-umaze-v1
    def __init__(self, inputargs):
        for k, v in inputargs.items():
            setattr(self, k, v)
        # Change logbase, horizon and n_diffusion_steps
        self.logbase = f'logs_dit_final_1M' #logs_dif_UMAZE_1M, logs_dit_final_1M
        self.horizon: int = 384 #384 for large, 256 for medium, 128 for small 
        self.n_diffusion_steps: int = 256 # 256 for large, 256 for medium, 64 for small
        self.savepath = f'{self.logbase}/{self.dataset}/diffusion/H{self.horizon}_T{self.n_diffusion_steps}'

args = argsmaker(config)

2024-06-07 19:24:15.852 | INFO     | __main__:<module>:3 - /root/DiffusionBasedRL
2024-06-07 19:24:15.854 | INFO     | __main__:<module>:5 - /opt/conda/envs/diffuser/bin/python


In [20]:
#-----------------------------------------------------------------------------#
#---------------------------------- dataset ----------------------------------#
#-----------------------------------------------------------------------------#

dataset_config = utils.Config(
    args.loader,
    savepath=(args.savepath, 'dataset_config.pkl'),
    env=args.dataset,
    horizon=args.horizon,
    normalizer=args.normalizer,
    preprocess_fns=args.preprocess_fns,
    use_padding=args.use_padding,
    max_path_length=args.max_path_length,
)

render_config = utils.Config(
    args.renderer,
    savepath=(args.savepath, 'render_config.pkl'),
    env=args.dataset,
)

dataset = dataset_config()
renderer = render_config()

observation_dim = dataset.observation_dim
action_dim = dataset.action_dim

[ utils/config ] Imported diffuser.datasets:GoalDataset

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/config ] Saved config to: logs_dit_final_1M/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl

[ utils/config ] Imported diffuser.utils:Maze2dRenderer

[utils/config ] Config: <class 'diffuser.utils.rendering.Maze2dRenderer'>
    env: maze2d-large-v1

[ utils/config ] Saved config to: logs_dit_final_1M/maze2d-large-v1/diffusion/H384_T256/render_config.pkl



load datafile: 100%|██████████| 8/8 [00:01<00:00,  7.05it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)


In [21]:

#-----------------------------------------------------------------------------#
#------------------------------ model & trainer ------------------------------#
#-----------------------------------------------------------------------------#

diffusion_config = utils.Config(
    args.diffusion,
    savepath=(args.savepath, 'diffusion_config.pkl'),
    horizon=args.horizon,
    observation_dim=observation_dim,
    action_dim=action_dim,
    n_timesteps=args.n_diffusion_steps,
    loss_type=args.loss_type,
    clip_denoised=args.clip_denoised,
    predict_epsilon=args.predict_epsilon,
    ## loss weighting
    action_weight=args.action_weight,
    loss_weights=args.loss_weights,
    loss_discount=args.loss_discount,
    device=args.device,
)

trainer_config = utils.Config(
    utils.Trainer,
    savepath=(args.savepath, 'trainer_config.pkl'),
    train_batch_size=args.batch_size,
    train_lr=args.learning_rate,
    gradient_accumulate_every=args.gradient_accumulate_every,
    ema_decay=args.ema_decay,
    sample_freq=args.sample_freq,
    save_freq=args.save_freq,
    label_freq=int(args.n_train_steps // args.n_saves),
    save_parallel=args.save_parallel,
    results_folder=args.savepath,
    bucket=args.bucket,
    n_reference=args.n_reference,
    n_samples=args.n_samples)

[ utils/config ] Imported diffuser.models:GaussianDiffusion

[utils/config ] Config: <class 'diffuser.models.diffusion.GaussianDiffusion'>
    action_dim: 2
    action_weight: 1
    clip_denoised: True
    horizon: 384
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 256
    observation_dim: 4
    predict_epsilon: False

[ utils/config ] Saved config to: logs_dit_final_1M/maze2d-large-v1/diffusion/H384_T256/diffusion_config.pkl


[utils/config ] Config: <class 'diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 50
    n_samples: 10
    results_folder: logs_dit_final_1M/maze2d-large-v1/diffusion/H384_T256
    sample_freq: 10000
    save_freq: 5000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: logs_dit_final_1M/maze2d-large-v1/diffusion/H384_T256/trainer_config.pkl



In [36]:
#-----------------------------------------------------------------------------#
#-------------------------------- instantiate --------------------------------#
#-----------------------------------------------------------------------------#

if 'DiT' in args.model:
    model = LDiT_models[args.model](
        in_channels = observation_dim + action_dim,
        max_in_len = args.horizon,
    )
    model = model.to(device=args.device)
else:
    model_config = utils.Config(
        args.model,
        savepath=(args.savepath, 'model_config.pkl'),
        horizon=args.horizon,
        transition_dim=observation_dim + action_dim,
        cond_dim=observation_dim,
        dim_mults=args.dim_mults,
        device=args.device,
    )
    model = model_config()


diffusion = diffusion_config(model)

trainer = trainer_config(diffusion, dataset, renderer)

In [37]:
checkpoints = sorted([int(x.split('_')[1].split('.')[0]) for x in os.listdir(trainer.logdir) if x.endswith('.pt')])
# TODO: Change this to 1M
checkpoints = [360000]

In [38]:
from diffuser.guides.policies import Policy

In [39]:
print(checkpoints[-1])
trainer.load(checkpoints[-1])
diffusion = trainer.model

360000


In [40]:
import diffuser.datasets as datasets
env = datasets.load_environment(args.dataset)

In [46]:
observation = env.reset()
print(observation)

[ 6.91533664  1.90086368 -0.04659669  0.05104924]


In [47]:
target = env._target
print(target)

(7, 9)


In [43]:
cond = {
    diffusion.horizon - 1: np.array([*target, 0, 0]),
}

In [44]:
## observations for rendering
rollout = [observation.copy()]


In [48]:
total_reward = 0

In [49]:
from loguru import logger

In [55]:
policy = Policy(diffusion, dataset.normalizer)
batch_size = 1

In [57]:
vis_freq = 10

In [60]:
for t in range(env.max_episode_steps):

        state = env.state_vector().copy()
        logger.info(f'state: {state}')

        ## can replan if desired, but the open-loop plans are good enough for maze2d
        ## that we really only need to plan once
        if t == 0:
            # logger.info(f'planning at t: {t}')
            cond[0] = observation

            action, samples = policy(cond, batch_size=batch_size)
            actions = samples.actions[0]
            sequence = samples.observations[0]
            # logger.info(f'actions: {actions}')
            # logger.info(f'samples: {samples}')
            # logger.info(f'sequence: {sequence}')
        # pdb.set_trace()

        # ####
        if t < len(sequence) - 1:
            next_waypoint = sequence[t+1]
        else:
            next_waypoint = sequence[-1].copy()
            next_waypoint[2:] = 0
            # pdb.set_trace()
        logger.info(f'next_waypoint: {next_waypoint}')
        ## can use actions or define a simple controller based on state predictions
        action = next_waypoint[:2] - state[:2] + (next_waypoint[2:] - state[2:])
        logger.info(f'action: {action}')
        # pdb.set_trace()
        ####

        # else:
        #     actions = actions[1:]
        #     if len(actions) > 1:
        #         action = actions[0]
        #     else:
        #         # action = np.zeros(2)
        #         action = -state[2:]
        #         pdb.set_trace()



        next_observation, reward, terminal, _ = env.step(action)
        logger.info(f'next_observation: {next_observation}')
        logger.info(f'reward: {reward}')
        logger.info(f'terminal: {terminal}')
        logger.info(f'_: {_}')
        total_reward += reward
        score = env.get_normalized_score(total_reward)
        print(
            f't: {t} | r: {reward:.2f} |  R: {total_reward:.2f} | score: {score:.4f} | ref_max_score: {env.ref_max_score} | ref_min_score: {env.ref_min_score}'
            f'{action}'
        )

        xy = next_observation[:2]
        goal = env.unwrapped._target
        print(f'maze | pos: {xy} | goal: {goal}')

        ## update rollout observations
        rollout.append(next_observation.copy())

        # logger.log(score=score, step=t)

        if terminal:
            break

        observation = next_observation


2024-06-07 09:04:46.352 | INFO     | __main__:<module>:4 - state: [ 6.91049418  1.90342088 -0.22499396  0.12603058]



                                                                                                    
1 / 256 [                                                            ]   0% | 156.6 Hz
t : 255
                                                                                                    
                                                                                                    
2 / 256 [                                                            ]   0% | 167.9 Hz
t : 254
                                                                                                    
                                                                                                    
3 / 256 [                                                            ]   1% | 173.4 Hz
t : 253
                                                                                                    
                                                                                                    
4 / 256

2024-06-07 09:04:47.748 | INFO     | __main__:<module>:27 - next_waypoint: [ 6.861636    1.93315    -0.36993265  0.10362434]
2024-06-07 09:04:47.749 | INFO     | __main__:<module>:30 - action: [-0.1937967   0.00732294]
2024-06-07 09:04:47.751 | INFO     | __main__:<module>:46 - next_observation: [ 6.90778804  1.90469563 -0.27061348  0.12747448]
2024-06-07 09:04:47.751 | INFO     | __main__:<module>:47 - reward: 0.0
2024-06-07 09:04:47.751 | INFO     | __main__:<module>:48 - terminal: False
2024-06-07 09:04:47.752 | INFO     | __main__:<module>:49 - _: {}
2024-06-07 09:04:47.753 | INFO     | __main__:<module>:4 - state: [ 6.90778804  1.90469563 -0.27061348  0.12747448]
2024-06-07 09:04:47.754 | INFO     | __main__:<module>:27 - next_waypoint: [ 6.8257637   1.9776955  -0.6233425   0.21815681]
2024-06-07 09:04:47.754 | INFO     | __main__:<module>:30 - action: [-0.43475338  0.16368218]
2024-06-07 09:04:47.755 | INFO     | __main__:<module>:46 - next_observation: [ 6.90405293  1.90635717 -

                                                                                                    
                                                                                                    
227 / 256 [#####################################################       ]  88% | 184.8 Hz
t : 29
                                                                                                    
                                                                                                    
228 / 256 [#####################################################       ]  89% | 184.7 Hz
t : 28
                                                                                                    
                                                                                                    
229 / 256 [#####################################################       ]  89% | 184.6 Hz
t : 27
                                                                                                    
     

2024-06-07 09:04:47.789 | INFO     | __main__:<module>:30 - action: [-0.83980513  0.17238616]
2024-06-07 09:04:47.791 | INFO     | __main__:<module>:46 - next_observation: [ 6.85300053  1.92210322 -1.28579579  0.34349936]
2024-06-07 09:04:47.791 | INFO     | __main__:<module>:47 - reward: 0.0
2024-06-07 09:04:47.791 | INFO     | __main__:<module>:48 - terminal: False
2024-06-07 09:04:47.792 | INFO     | __main__:<module>:49 - _: {}
2024-06-07 09:04:47.793 | INFO     | __main__:<module>:4 - state: [ 6.85300053  1.92210322 -1.28579579  0.34349936]
2024-06-07 09:04:47.793 | INFO     | __main__:<module>:27 - next_waypoint: [ 6.737054   1.9640894 -2.0685525  0.6761017]
2024-06-07 09:04:47.795 | INFO     | __main__:<module>:30 - action: [-0.89870337  0.37458849]
2024-06-07 09:04:47.796 | INFO     | __main__:<module>:46 - next_observation: [ 6.83803281  1.92642217 -1.49677213  0.43189471]
2024-06-07 09:04:47.796 | INFO     | __main__:<module>:47 - reward: 0.0
2024-06-07 09:04:47.796 | INFO   

...

t: 691 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.81206794e-04 6.80149796e+00]
maze | pos: [7.00525964 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 692 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.79352115e-04 6.80149796e+00]
maze | pos: [7.0052058  2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 693 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.77516418e-04 6.80149796e+00]
maze | pos: [7.00515252 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 694 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.75699510e-04 6.80149796e+00]
maze | pos: [7.00509978 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, spar

2024-06-07 09:04:51.842 | INFO     | __main__:<module>:4 - state: [ 7.00355773e+00  2.19850204e+00 -3.67904456e-03  6.29734138e-15]
2024-06-07 09:04:51.842 | INFO     | __main__:<module>:27 - next_waypoint: [7. 9. 0. 0.]
2024-06-07 09:04:51.843 | INFO     | __main__:<module>:30 - action: [1.21317490e-04 6.80149796e+00]
2024-06-07 09:04:51.845 | INFO     | __main__:<module>:46 - next_observation: [ 7.00352131e+00  2.19850204e+00 -3.64138897e-03  6.29734138e-15]
2024-06-07 09:04:51.846 | INFO     | __main__:<module>:47 - reward: 0.0
2024-06-07 09:04:51.847 | INFO     | __main__:<module>:48 - terminal: False
2024-06-07 09:04:51.847 | INFO     | __main__:<module>:49 - _: {}
2024-06-07 09:04:51.848 | INFO     | __main__:<module>:4 - state: [ 7.00352131e+00  2.19850204e+00 -3.64138897e-03  6.29734138e-15]
2024-06-07 09:04:51.849 | INFO     | __main__:<module>:27 - next_waypoint: [7. 9. 0. 0.]
2024-06-07 09:04:51.850 | INFO     | __main__:<module>:30 - action: [1.20075786e-04 6.80149796e+00]


sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 730 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.21317490e-04 6.80149796e+00]
maze | pos: [7.00352131 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 731 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.20075786e-04 6.80149796e+00]
maze | pos: [7.00348527 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 732 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.18846792e-04 6.80149796e+00]
maze | pos: [7.0034496  2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 733 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[1.17630377e-04 6.80149796e

2024-06-07 09:04:52.046 | INFO     | __main__:<module>:4 - state: [ 7.00240652e+00  2.19850204e+00 -2.48858204e-03  6.29734138e-15]
2024-06-07 09:04:52.049 | INFO     | __main__:<module>:27 - next_waypoint: [7. 9. 0. 0.]
2024-06-07 09:04:52.049 | INFO     | __main__:<module>:30 - action: [8.20616658e-05 6.80149796e+00]
2024-06-07 09:04:52.050 | INFO     | __main__:<module>:46 - next_observation: [ 7.00238189e+00  2.19850204e+00 -2.46311101e-03  6.29734138e-15]
2024-06-07 09:04:52.051 | INFO     | __main__:<module>:47 - reward: 0.0
2024-06-07 09:04:52.051 | INFO     | __main__:<module>:48 - terminal: False
2024-06-07 09:04:52.052 | INFO     | __main__:<module>:49 - _: {}
2024-06-07 09:04:52.053 | INFO     | __main__:<module>:4 - state: [ 7.00238189e+00  2.19850204e+00 -2.46311101e-03  6.29734138e-15]
2024-06-07 09:04:52.053 | INFO     | __main__:<module>:27 - next_waypoint: [7. 9. 0. 0.]
2024-06-07 09:04:52.054 | INFO     | __main__:<module>:30 - action: [8.12217519e-05 6.80149796e+00]


t: 767 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[8.29102653e-05 6.80149796e+00]
maze | pos: [7.00240652 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 768 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[8.20616658e-05 6.80149796e+00]
maze | pos: [7.00238189 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 769 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[8.12217519e-05 6.80149796e+00]
maze | pos: [7.00235751 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, sparse is getting called
0.0, sparse, ... is getting called
t: 770 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[8.03904347e-05 6.80149796e+00]
maze | pos: [7.00233338 2.19850204] | goal: (7, 9)
sparse, mess it up
0.0, sparse, spar